In [163]:
import json
import pandas as pd
import numpy as np

# import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

# nltk.download("punkt")

import re
from collections import defaultdict

In [116]:
ENGSTOP = stopwords.words('english')


# 0. peek 2 json objects

In [2]:
# !head -n 2 sample_from_full.json

In [3]:
# b = db.read_text('2008.json').map(json.loads)
# b

In [4]:
cols_needed = ['patentCaseMetadata.applicationNumberText.value',
               'patentCaseMetadata.applicationNumberText.electronicText',
               'patentCaseMetadata.filingDate',
               'patentCaseMetadata.applicationTypeCategory',
               'patentCaseMetadata.inventionTitle.content',
               'patentCaseMetadata.groupArtUnitNumber.value',
               'patentCaseMetadata.groupArtUnitNumber.electronicText',
               'patentCaseMetadata.applicationConfirmationNumber',
               'patentCaseMetadata.firstInventorToFileIndicator',
               'patentCaseMetadata.inventionTitle.content',
               'patentCaseMetadata.applicationStatusCategory',
               'patentCaseMetadata.applicationStatusDate',
               'patentCaseMetadata.officialFileLocationCategory',
               'patentCaseMetadata.patentClassificationBag.cpcClassificationBagOrIPCClassificationOrECLAClassificationBag',
               'patentCaseMetadata.patentGrantIdentification.patentNumber',
               'patentCaseMetadata.patentGrantIdentification.grantDate']


In [176]:
def read_text(fp):
    with open(fp, 'r') as file:
        for i in file:
            yield i

def clean_df(df):
    out = []
    for col in cols_needed:
        try:
            to_append = df.loc[:, col].values[0]
            out.append(to_append)
        except:
            out.append(np.NaN)
    return out

def str2json(json_str):
    json_file = json.loads(json_str)
    
    return json_file

def json2df(json_file):
    norm = pd.json_normalize(json_file)
    out = clean_df(norm)
#     print(out)
    return out


def json_parser(json_path, num_iter = 1):
    idx = 0
    out = []
    with open('meta_data.csv', 'a') as meta_data:
        for i in read_text(json_path):
    #         print(i)
            if idx == 0:
                idx += 1
                continue
            elif idx > num_iter:
                break
            else: 
                if i[0] == ',':
                    json_str = i[1:]
                else:
                    json_str = i
    #             print(json_str)
                out = json2df(str2json(json_str))
                to_write = str(out)[1:-1]
                print(to_write)
            meta_data.write(to_write + '\n')
            idx += 1
            

# 1. call this cell given __num__ of iterations to output the cleaned csv
- TODO further clean the nested list; get the nested values
- TODO: create a header for the csv

# 2. bag of words

In [170]:
# json_parser('2008.json', 10)
# !head -n 1 sample_from_full.json


In [171]:
# detailedDescription = get_field_values(data, 'abstract')
# detailedDescription
# json.loads(data)['abstract']
# frequency_dict(detailedDescription)

In [ ]:
def get_label(data):
    PATTERN = "applicationStatusCategory\":\"([\w+|\d+|\s|-]+)"
    return re.findall(PATTERN, data)
            
def get_field_values(data, field):
    # 'abstract', 'detailedDescription', 'summaryOfInvention'
    # [para.values() for para in json.loads(data)['detailedDescription']
    out = ''
    try:
        for val in json.loads(data)[field]:
    #         print(val)
            try:
                out += val['text'].lower()
            except:
                out += val[0].lower()
    except:
        return out
#         print('no %s found'  %field)
    return out

def frequency_dict(text, out , stopwords = ENGSTOP):
    
    words = word_tokenize(text)
    stemmer = PorterStemmer()

    for w in words:
        word_stem = stemmer.stem(w)
        if (word_stem not in stopwords) and (word_stem.isalnum()):
            out[word_stem] += 1
    
    return out



# build bag of words - write all text set to local disk

In [172]:
with open('full_text.txt', 'a') as full_text_file:
    
    for data in read_text('sample_from_full.json'):
    #     print(data)    
        label = get_label(data)
        if len(label) == 1:
            UNIQUE_LABELS[label[0]] += 1
            valid += 1


            abstract = get_field_values(data, 'abstract')
            detailedDescription = get_field_values(data, 'detailedDescription')
            summaryOfInvention = get_field_values(data, 'summaryOfInvention')
            full_text = abstract + ' ' + detailedDescription + ' ' + summaryOfInvention + '\n'
            full_text_file.write(full_text)

        elif len(label) == 0:
            err += 1
    

# build bag of words - preform frequency_dict

In [181]:
bags = defaultdict(int)
for line in read_text('full_text.txt'):
    bags = frequency_dict(line, bags)
bags    

KeyboardInterrupt: 

In [238]:
bag_of_words = sorted(bags.items(), key= lambda x: x[1], reverse=True)[:1000]
bag_of_words = [w[0] for w in bag_of_words]
bag_of_words

['may',
 'embodi',
 'one',
 'devic',
 'includ',
 'exampl',
 'use',
 'first',
 '1',
 'fig',
 '2',
 'second',
 'data',
 'system',
 'control',
 'provid',
 'thi',
 'process',
 'present',
 'form',
 'signal',
 'portion',
 'layer',
 'user',
 'unit',
 '3',
 'invent',
 'inform',
 'method',
 'oper',
 'gener',
 'describ',
 'imag',
 'compris',
 'least',
 'wa',
 'base',
 'configur',
 'also',
 'surfac',
 'time',
 'b',
 'ani',
 'receiv',
 'posit',
 'ha',
 'determin',
 'refer',
 '10',
 'connect',
 'accord',
 'shown',
 'materi',
 'commun',
 'element',
 '4',
 'perform',
 'comput',
 'group',
 'display',
 'step',
 'cell',
 'compon',
 'valu',
 'c',
 'select',
 'differ',
 'set',
 'prefer',
 '5',
 'illustr',
 'applic',
 '100',
 'modul',
 'network',
 'acid',
 'direct',
 'structur',
 'light',
 'addit',
 'power',
 'end',
 'line',
 '1h',
 'region',
 'within',
 'number',
 'member',
 'anoth',
 'function',
 'two',
 'abov',
 'part',
 'herein',
 '20',
 'side',
 'correspond',
 'output',
 'indic',
 'r',
 'memori',
 '6'

# build bag of words

In [245]:
def label2int(lab):
    return LABEL_MAPPING[lab]

def vectorizer(input_text, bag_of_words):
    
    vec = []
    for w in bag_of_words: 
        if w in word_tokenize(input_text): 
            vec.append('1') 
        else: 
            vec.append('0') 
    return vec
            
# def term_frequency():

# def tf_idf():
    
    

In [248]:
LABEL_MAPPING = {
    'Patent Expired Due to NonPayment of Maintenance Fees Under 37 CFR 1': '1',
    'Patented Case': '1',
    'Final Rejection Mailed': '0',
    'Expressly Abandoned  --  During Publication Process': '0',
    'Expressly Abandoned  --  During Examination': '0',
    'Abandonment for Failure to Correct Drawings': '0',
    'Abandoned  --  Incomplete Application ': '0',
    'Abandoned  --  Failure to Respond to an Office Action': '0',
    'Abandoned  --  Failure to Pay Issue Fee': '0',
    'Abandoned  --  After Examiner': '0'

}

In [254]:
features  = []
for data in read_text('sample_from_full.json'):
    #     print(data)    
    label = get_label(data)
    label_candi = LABEL_MAPPING.keys()
    
    if (len(label) == 1) and (label[0] in label_candi):

        abstract = get_field_values(data, 'abstract')
        detailedDescription = get_field_values(data, 'detailedDescription')
        summaryOfInvention = get_field_values(data, 'summaryOfInvention')
        full_text = abstract + ' ' + detailedDescription + ' ' + summaryOfInvention + '\n'
        
        feature_vec = vectorizer(full_text, bag_of_words)
        
        valid_lab = label2int(label[0])
        
        
        with open('vectorized.txt', 'a') as fh:
            line =  ','.join(feature_vec) + ',' + valid_lab + '\n'
            fh.write(line)
#             break

KeyboardInterrupt: 

In [256]:
df = pd.read_csv('vectorized.txt', header  = None, sep = ',')
df

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
0,1,0,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,0,1,0,0,0,1,1,1,1,...,0,1,0,0,0,0,0,0,0,1
2,1,0,1,0,0,0,0,1,1,1,...,0,0,0,1,0,0,0,0,0,1
3,1,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1
4,1,0,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
5,1,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
6,1,0,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
7,1,0,1,0,0,0,1,1,1,1,...,0,0,0,1,1,0,0,0,1,1
8,1,0,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0
9,1,0,1,0,0,0,1,0,1,1,...,0,1,0,0,0,0,0,0,0,0
